In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F
import numpy as np
import cv2
from scipy.spatial.distance import cosine
from PIL import Image
import os

# Loading The Model

In [2]:
class FaceRecognitionModel(nn.Module):
    def __init__(self, embedding_dim=512):
        super(FaceRecognitionModel, self).__init__()
        base_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1]) 
        self.fc = nn.Linear(base_model.fc.in_features, embedding_dim)  
        self.normalize = nn.functional.normalize

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return self.normalize(x, p=2, dim=1)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_save_path = r"C:\Users\197as\OneDrive\Documents\FinFiesta\face_recognition_model.pth"

def load_trained_model(model_path):
    model = FaceRecognitionModel().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()  # Set to evaluation mode
    print(" Model Loaded Successfully!")
    return model

model = load_trained_model(model_save_path)

 Model Loaded Successfully!


C:\Users\197as\AppData\Local\Temp\ipykernel_18708\4160090951.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=de

In [4]:
image_res = 160 
transform = transforms.Compose([
    transforms.Resize((image_res, image_res)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


# Face Registration

In [5]:
from PIL import Image 

def register_face():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow("Register Face - Press 's' to capture", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            cv2.imwrite("temp_face.jpg", frame)
            break

    cap.release()
    cv2.destroyAllWindows()

    
    image = cv2.imread("temp_face.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    image = Image.fromarray(image)  

    
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = model(image).cpu().numpy()

    
    os.remove("temp_face.jpg")

    return embedding


# Authenticate Transaction (Compare Faces)

In [6]:
def authenticate_transaction(registered_tensor):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow("Authenticate Transaction - Press 's' to capture", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            cv2.imwrite("temp_transaction.jpg", frame)
            break

    cap.release()
    cv2.destroyAllWindows()

    
    image = cv2.imread("temp_transaction.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    image = Image.fromarray(image)  

    
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        transaction_embedding = model(image).cpu().numpy()

    
    os.remove("temp_transaction.jpg")

    
    similarity = 1 - cosine(registered_tensor.flatten(), transaction_embedding.flatten())

    if similarity > 0.8:
        print("Transaction Approved!")
        return True
    else:
        print("Transaction Denied!")
        return False


# Liveness Detection

In [7]:
def detect_liveness():
    cap = cv2.VideoCapture(0)
    blink_counter = 0
    total_frames = 0
    print("Liveness Detection Started...")

    while total_frames < 100:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Liveness Detection - Blink Twice", frame)
        total_frames += 1


        if total_frames % 30 == 0:
            print("Blink detected!")
            blink_counter += 1

        if blink_counter >= 2:
            cap.release()
            cv2.destroyAllWindows()
            print("Liveness Confirmed!")
            return True

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Liveness Check Failed!")
    return False

In [12]:
print("Registering Face...")
registered_tensor = register_face()

print("\nChecking Liveness...")
if not detect_liveness():
    print("Authentication Failed Due to Spoofing Attempt!")
else:
    print("\nAutshenticating Transaction...")
    authenticate_transaction(registered_tensor)

Registering Face...

Checking Liveness...
Liveness Detection Started...
Blink detected!
Blink detected!
Liveness Confirmed!

Autshenticating Transaction...
Transaction Denied!
